<h1><center> Practical assignment # 2: convolutional neural networks </center></h1>

1. **Start date**: 12 Apr 2017 (Lecture # 7)
2. **Deadline**: 24 May 2017

### General information on the task:
**Diabetic retinopathy** is a common eye disease affecting more than 93 mln people worldwide and notorious for being one of the major causes of blindness in the working-age population of the developed world. Scientists distinguish 4 stages of the disease differing in their symptoms and influence on human's health. Pregression to vision impairment can be slowed or even averted if diabetic retinopathy is detected in time. However, this can be difficult as the disease often shows few symptoms (stages 1, 2) until it is too late to provide effective treatment (stage 4). Currently, detecting diabetic retinopathy is a time-consuming and manual process that requires a trained clinician to examine and evaluate digital color fundus photos of the retina: 

<tr>
 <td><img src='good.jpeg' width="180"/></td>  stage 0:
 <td><img src='bad.jpeg' width="180"/></td>   stage 4:
</tr>

In 2015, [EyePACS](http://www.eyepacs.com/)  and [California HealthCare Foundation](http://www.chcf.org/) organized a [competition](https://www.kaggle.com/c/diabetic-retinopathy-detection) among data scientists on Kaggle platform. The main goal of this competition was to create a model capable of identifying diabetic retinopathy stage given a color fundus photography as input. To evaluate the quality of the model, a metric called [quadratic weighted kappa](https://www.kaggle.com/c/diabetic-retinopathy-detection#evaluation) was suggested. Quadratic weighted kappa measures correlation between predicted and correct values. The more random the prediction is, the closer kappa is to zero. In contrast, the stronger the positive correlation is, the closer kappa is to unit. Top-10 participants were able to achieve kappa greater than 0.8, the level of performance comparable with that of clinician. 

Inspired by the results of this competition, we decided to propose you a similar task as your 2 assignment. 

### Assignment objectives:

1. Create a convolutional neural network that takes color fundus images as input and predicts the stage of diabetic retinopathy on them.  
2. Assignment will be considered as successfully completed if either quadratic weighted kappa for predictions of your model on test dataset is geater than 0.35 or your results are in TOP-40% of the classroom final rating 

### Datasets: 

This assignment uses preprocessed images of color fundus from [Kaggle competition datasets](https://www.kaggle.com/c/diabetic-retinopathy-detection/data). Preprocessing includes blurring, cropping, and resizing to 512x512. To prevent students from augmenting their training set by publicly available Kaggle test dataset, some color distortions were introduced to the images. These distortions cause funny changes in color palitra, but do not affect training efficiency. Numbers in filenames are ids of people. Left and Right mean left and right eyes (usually there is a strong correlation between stages of two eyes)  

1. Training dataset consists of 35126 images each of which is assigned to one of 5 classes (0 class means a healthy eye). We have already divided training dataset into train and validation parts, but feel free to make your own division. Dataset is available [here](https://yadi.sk/d/mzaFYDgG3GtdV3). Table containing names of files and disease stage per each of the images in .csv format can be taken [here](https://yadi.sk/d/7SFlJS9I3GtgGD)

2. Test dataset is composed of 53576 unclassified images and can be downloaded [here](https://yadi.sk/d/HkdQMqoI3GtcQa)    

### Submitting results:
After Lecture # 8, you will be able to submit your predictions for test dataset images. Predictions should be sent in .csv file with two columns (image: filename without '.jpeg', level: integer number of predicted class). Submission details will be discussed later. 

In [1]:
import pandas as pd
prediction = pd.read_csv('trainLabels.csv', nrows=3)
print prediction

      image  level
0   10_left      0
1  10_right      0
2   13_left      0


## Baseline model

To give you a basic idea of how to approach the assignment, we have prepared a simple baseline model. Below, we guide you through the key steps of its pipeline

1. Download the model and the data. Check that (i) folders data (with subfolders train, test and validation), models and model_params (currently, empty) are in the assignment's main directory, (ii) scripts train.py and evaluate.py as well as .csv file ['trainLabels.csv'](https://yadi.sk/d/7SFlJS9I3GtgGD) are in the main directory, (iii) scripts settings.py (with model's hyperparameters), utils.py, and model.py are in models folder. 
2. Train the model by launching 'python train.py' in the main directory 
3. While training, you will be able to observe training statistics (train_loss, train_acc, val_loss, val_acc) that after finishing the learning process, will be saved in 'history.csv' file. Weights for best models are stored in model_params folder and acquire names refering to their val_acc and epoch when they were recorded.
4. Now you can calculate quadratic weighted kappa for several good models on validation dataset. Below, we demonstrate how to do it by using code from evaluate.py.
5. Take your best model and predict classes for test dataset. Send your predictions to us, as it was described in 'Submitting results'. We took checkpoint weights-improvement-35-0.61.hdf5 for predicting labels of test dataset and obtained kappa = 0.16

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from models.settings import *
from keras.models import model_from_json
from models.utils import quadratic_weighted_kappa

Using TensorFlow backend.


In [3]:
# test data dir
test_data_dir = VALIDATION_DATA_DIR

# read names of images and correct labels for them
correct_data = pd.read_csv('trainLabels.csv')

# create pool of images to be evaluated
image_file_pool = []
image_name_pool = []
class_names = [name for name in os.listdir(test_data_dir)]
for class_name in class_names:
    for file in os.listdir(test_data_dir+class_name+'/'):
        if file.endswith(".jpeg"):
            image_file_pool.append(test_data_dir+class_name+'/'+file)
            image_name_pool.append(file[:-5])

In [ ]:
execfile('train_last_layers.py')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0         
__________

In [ ]:
#a = os.walk(WEIGHTS_DIR)
#list_of_checkpoints = list(a)[0][-1]
for checkpoint in ["weights-improvement-01-0.53.hdf5"]:
    print(checkpoint)
    # load json and create model
    #json_file = open('model.json', 'r')
    #loaded_model_json = json_file.read()
    #json_file.close()

    # load model's weights
    model.load_weights(WEIGHTS_DIR + checkpoint)
    
    predicted_label = []
    correct_label = []

    # predict and store labels
    for image_idx in range(len(image_file_pool)):
        img_array = np.array(Image.open(image_file_pool[image_idx]))
        img_array = img_array[None, :]
        predicted_label.append(np.argmax(model.predict(img_array).ravel()))
        correct_label.append(correct_data['level'].loc[correct_data['image'] == image_name_pool[image_idx]].tolist()[0])

    print 'checkpoint {0}: {1:5.4f}'.format(checkpoint, quadratic_weighted_kappa(predicted_label, correct_label))

weights-improvement-01-0.53.hdf5


In [5]:
execfile('train_all.py')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0         
__________

KeyboardInterrupt: 

If you want to plot accuracy chart with quadratic weighted kappa data for each epoch, you need to save your model each epoch (in model.py, change save_best_only=True to False in model_checkpoint) and add names of saved models to list_of_checkpoints above. Then you can augment 'history.csv' file with these data and obtain something like our accuracy chart for the baseline model:

In [ ]:
acc_history.tail()

In [ ]:
import matplotlib.pyplot as plt
acc_history = pd.read_csv('history.csv', index_col=0)
plt.plot(acc_history['val_acc'], color='b', label='val acc')
plt.plot(acc_history['acc'], color='g', label='train acc')
plt.legend(fontsize=12)
plt.show()

In [ ]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

In [ ]:
from models.simple_model import initialize_model
from models.settings import *

In [ ]:
from __future__ import print_function
from scipy.misc import imsave
import numpy as np
import time
import keras
from keras import backend as K
import matplotlib.pyplot as plt

# dimensions of the generated pictures for each filter.
img_width = IMG_WIDTH
img_height = IMG_HEIGHT

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'conv2d_4'

# load model
loc_json = 'model.json'
loc_h5 = 'model_params/weights-improvement-04-0.39.hdf5'

with open(loc_json, 'r') as json_file:
    loaded_model_json = json_file.read()

#model = keras.models.model_from_json(loaded_model_json)
model = initialize_model()

# load weights into new model
model.load_weights(loc_h5)
print('Model loaded.')

model.summary()

In [ ]:
def visualize_filters(model, layer_name, n=16):
    # this is the placeholder for the input images
    #input_img = model.input

    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    for x in layer_dict:
        print(x)

    kept_filters = []
    for filter_index in range(16):
        # we only scan through the first 200 filters,
        # but there are actually 512 of them
        print('Processing filter %d' % filter_index)
        start_time = time.time()

        # we build a loss function that maximizes the activation
        # of the nth filter of the layer considered
        layer_input = layer_dict[layer_name].input
        input_img = layer_input
        layer_output = layer_dict[layer_name].output
        if K.image_dim_ordering() == 'th':
            loss = K.mean(layer_output[:, filter_index, :, :])
        else:
            loss = K.mean(layer_output[:, :, :, filter_index])


        # we compute the gradient of the input picture wrt this loss
        grads = K.gradients(loss, input_img)[0]

        # normalization trick: we normalize the gradient
        grads = normalize(grads)

        # this function returns the loss and grads given the input picture
        iterate = K.function([input_img], [loss, grads])

        # step size for gradient ascent
        step = 1.

        # we start from a gray image with some random noise
        img_shape = (1, 127, 127,  16)
        print(img_shape)
        input_img_data = np.random.random(img_shape)
        input_img_data = (input_img_data - 0.5) * 20 + 128

        # we run gradient ascent for 20 steps
        for i in range(20):
            loss_value, grads_value = iterate([input_img_data])
            input_img_data += grads_value * step

            print('Current loss value:', loss_value)
            if loss_value <= 0.:
                # some filters get stuck to 0, we can skip them
                break

        # decode the resulting input image
        #if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
        end_time = time.time()
        print('Filter %d processed in %ds' % (filter_index, end_time - start_time))
        
    # we will stich the best 64 filters on a 8 x 8 grid.
    n = 4

    # the filters that have the highest loss are assumed to be better-looking.
    # we will only keep the top 64 filters.
    kept_filters.sort(key=lambda x: x[1], reverse=True)
    kept_filters = kept_filters[:n * n]

    # build a black picture with enough space for
    # our 8 x 8 filters of size 128 x 128, with a 5px margin in between
    margin = 5
    width = n * img_width + (n - 1) * margin
    height = n * img_height + (n - 1) * margin
    stitched_filters = np.zeros((width, height, 3))

    # fill the picture with our saved filters
    for i in range(n):
        for j in range(n):
            img, loss = kept_filters[i * n + j]
            stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                             (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

    # save the result to disk
    imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)
    plt.figure(figsize=(15, 15))
    plt.imshow(stitched_filters)
    plt.show()

In [ ]:
visualize_filters(model, 'conv2d_2')